# Model1) 2-way interaction between mutation and CNAs within a gene-tissue pair. 

In [3]:
##########################
library(stringr); library(reshape2); library(data.table); library(dplyr);
# library(viridis);


# memory.limit(size = 24000)

# setwd("~/Documents/2way_input/")
# outputDirectory='~/Documents/output/' #directory for output files

`%+%` = function(a, b) paste0(a, b)
clipOut = function( x ) { write.table(x,"clipboard-16384", sep="\t", row.names=F, quote=F) }
clipIn = function( ) { read.table("clipboard", header=TRUE, na.strings=c("NA","NaN","?"), sep="\t"); }
numNas = function( dt ) { sapply( pcaPointsForIndiv, function(x){sum(is.na(x))} )   }
cosineDist <- function(x){ as.dist(1 - x%*%t(x)/(sqrt(rowSums(x^2) %*% t(rowSums(x^2))))) }
mbind<-function(...){
    Reduce( function(x,y){cbind(x,y[match(row.names(x),row.names(y)),])}, list(...) )
}

#########################################################
#Model1: 2-way interactions between mutation and CNAs within a cancer gene across cancer types

loadInputData = function( fn  ) {
    
    inData = fread(fn, data.table = T)  # glimpse(inData)
    
    inData = inData[, colnames(inData) %>% str_subset( "(Mut|NoMut)_(Del|Loss|WT|Gain|Amp)$") %>% c("Gene","Tissue", .), with=F ]
    
    inData =
    inData %>% melt( id.vars=c("Gene", "Tissue"), variable.name="variantType", value.name="N" ) %>%
    mutate( mut = c("Mut"=1, "NoMut"=0)[ str_match( variantType, "([^_]+)_[^_]+" )[,2] ] ) %>%
    mutate( cna = c("Loss"=-1, "WT"=0, "Gain"=1)[ str_match( variantType, "[^_]+_([^_]+)")[,2] ] ) %>%
    select(-variantType)
    return(inData)
}


#input_file name: (1) Obs_LLR_randomization_AcrossCancer_mutation_Tissue_Loss.txt, (2) Obs_LLR_randomization_AcrossCancer_mutation_Tissue_Gain.txt
# When the number of sample is zero at least one of the sample classes, 1 could be added to each frequency when running regression model.

Input_Input_direc <- paste('~/data/Cancer_ML/cancerfitness_suhancho_loss.tsv', sep='\t')

Input.dataLong = loadInputData( Input_Input_direc)
Input.data = Input.dataLong %>% reshape2::dcast( Gene+Tissue ~ mut+cna, value.var="N") ## updated because of error message

cat(sprintf( "%s\n", Input_Input_direc ));

Input.data <- fread(file = Input_Input_direc, data.table = F)
Input.data <- Input.data[,c(1:2,3:6)]

names(Input.data) <- gsub(x = names(Input.data), pattern = "NoMut", replacement = 0) %>%
gsub(pattern = "Mut", replacement = 1) %>%
gsub(pattern = "Loss", replacement = -1) %>%
gsub(pattern = "WT", replacement = 0) %>%
gsub(pattern = "Gain", replacement = 1) 

Input.data <- as.data.table(Input.data)

Input.dataLlm = copy(Input.dataLong)
Input.dataLlm <- as.data.table(Input.dataLlm)

Input.dataLlm[, mut:=as.factor(mut)]
Input.dataLlm[, cna:=as.ordered(cna)]
Input_results <- c()
#####################
cat(sprintf( "Tissue %s\n", 'HNSC' ));
for ( aGene in Input.dataLlm[, unique(Gene)] %>% setdiff("_allCancerGenes") ) {

    aLlm <- glm(N ~ mut + cna + mut:cna, family=poisson(link="log"),
    control=glm.control(epsilon = 1e-6,maxit=100), data=Input.dataLlm[Gene == aGene])

    aLlCoef = summary(aLlm) %>% coef
    row.names(aLlCoef)[1] <- paste('intercept_',aGene,sep='')
    Input_results <-Input_results %>%rbind(aLlCoef)
}

Input_results <- as.data.frame(Input_results)
write.table(Input_results %>% mutate(Tissue=tumor,variable=row.names(Input_results)),
file= paste(outputDirectory,"Model1_Obs_", tumor,"_Loss.txt",sep=''),
quote=FALSE, sep='\t',row.names=F,col.names = T)


Warning message in melt.data.table(., id.vars = c("Gene", "Tissue"), variable.name = "variantType", :
“'measure.vars' [Mut_Loss, NoMut_Loss, Mut_WT, NoMut_WT, ...] are not all of the same type. By order of hierarchy, the molten data value column will be of type 'double'. All measure variables not of type 'double' will be coerced too. Check DETAILS in ?melt.data.table for more on coercion.”


~/data/Cancer_ML/cancerfitness_suhancho_loss.tsv
Tissue HNSC


In [7]:
Input_results <- as.data.frame(Input_results)
write.table(Input_results %>% mutate(Tissue='HNSC',variable=row.names(Input_results)),
file= '~/data/Cancer_ML/cancerfitness_suhancho_loss_result.tsv',
quote=FALSE, sep='\t',row.names=F,col.names = T)


In [10]:
r = read.table('~/data/Cancer_ML/cancerfitness_suhancho_loss_result.tsv',sep='\t',header=1)

In [15]:
r[(grep('MIMAT',r$variable)),]

Estimate,Std..Error,z.value,Pr...z..,Tissue,variable
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>


In [29]:

mir = r[grep('MIR',r$variable),]


In [31]:
mir

,Estimate,Std..Error,z.value,Pr...z..,Tissue,variable
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
41,5.317350,0.05604144,94.88247,0.000000e+00,HNSC,intercept_MIR525
77,5.115117,0.06226402,82.15206,0.000000e+00,HNSC,intercept_MIR409
201,5.430268,0.05110154,106.26428,0.000000e+00,HNSC,intercept_MIR1269B
341,5.607960,0.04286080,130.84125,0.000000e+00,HNSC,intercept_MIR100
373,5.144732,0.06710876,76.66260,0.000000e+00,HNSC,intercept_MIR5188
449,5.627518,0.04402340,127.83014,0.000000e+00,HNSC,intercept_MIR566
489,4.597816,0.10136575,45.35868,0.000000e+00,HNSC,intercept_MIR3943
533,5.156622,0.06083382,84.76571,0.000000e+00,HNSC,intercept_MIR208B
577,3.489573,0.20751702,16.81584,1.868318e-63,HNSC,intercept_MIR1207
